In [ ]:
# ## install finrl library
# !pip install wrds
# !pip install swig
# !pip install yfinance==0.2.44
# !pip install -q condacolab
# import condacolab
# condacolab.install()
# !apt-get update -y -qq && apt-get install -y -qq cmake libopenmpi-dev python3-dev zlib1g-dev libgl1-mesa-glx swig
# !pip install git+https://github.com/flpymonkey/FinRL_Online_Portfolio_Benchmarks.git

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime

%matplotlib inline
from finrl.config_tickers import DOW_30_TICKER
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent,DRLEnsembleAgent

from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline

from pprint import pprint

# import sys
# sys.path.append("../FinRL-Library")

import itertools

In [4]:
import os
from finrl.main import check_and_make_directories
from finrl.config import (
    DATA_SAVE_DIR,
    TRAINED_MODEL_DIR,
    TENSORBOARD_LOG_DIR,
    RESULTS_DIR,
    INDICATORS,
    TRAIN_START_DATE,
    TRAIN_END_DATE,
    TEST_START_DATE,
    TEST_END_DATE,
    TRADE_START_DATE,
    TRADE_END_DATE,
)

check_and_make_directories([DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR])

In [5]:
# TODO, remove DOW from training as it is missing some values in 2019
print(DOW_30_TICKER)

['AXP', 'AMGN', 'AAPL', 'BA', 'CAT', 'CSCO', 'CVX', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'KO', 'JPM', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PG', 'TRV', 'UNH', 'CRM', 'VZ', 'V', 'WBA', 'WMT', 'DIS', 'DOW']


In [6]:
TRAIN_START_DATE = '2009-04-01'
TRAIN_END_DATE = '2021-01-01'
TEST_START_DATE = '2021-01-01'
TEST_END_DATE = '2024-01-01'

from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.config_tickers import DOW_30_TICKER


# TODO Drop the DOW stock
value_to_remove = "DOW"
# Create a new list without the specified string
DOW_30_TICKER = [x for x in DOW_30_TICKER if x != value_to_remove]
print(DOW_30_TICKER)

TIME_WINDOW = 40
COMMISSION_FEE_PERCENT = 0.001
INITIAL_CASH = 1000000

# TODO currently an issue with yfiance
# df = YahooDownloader(start_date = TRAIN_START_DATE,
#                      end_date = TEST_END_DATE,
#                      ticker_list = DOW_30_TICKER).fetch_data()

df_list = []

for ticker in DOW_30_TICKER:
    # Fetch data for each ticker
    portfolio_raw_df = YahooDownloader(start_date=TRAIN_START_DATE,
                                       end_date=TEST_END_DATE,
                                       ticker_list=[ticker]).fetch_data()
    # Append the fetched DataFrame to the list
    df_list.append(portfolio_raw_df)

# Concatenate all DataFrames row-wise
portfolio_raw_df = pd.concat(df_list, axis=0, ignore_index=True)
df = portfolio_raw_df
# Display the concatenated DataFrame
print(portfolio_raw_df)

['AXP', 'AMGN', 'AAPL', 'BA', 'CAT', 'CSCO', 'CVX', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'KO', 'JPM', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PG', 'TRV', 'UNH', 'CRM', 'VZ', 'V', 'WBA', 'WMT', 'DIS']


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Shape of DataFrame:  (3020, 8)
Shape of DataFrame:  (3020, 8)



[*********************100%***********************]  1 of 1 completed


Shape of DataFrame:  (3020, 8)


...

Shape of DataFrame:  (3020, 8)
Shape of DataFrame:  (3020, 8)



[*********************100%***********************]  1 of 1 completed


Shape of DataFrame:  (3020, 8)


[*********************100%***********************]  1 of 1 completed

Shape of DataFrame:  (3020, 8)
             date       open       high        low      close    volume  tic  \
0      2011-01-03  43.299999  43.619999  43.110001  35.531181   7633300  AXP   
1      2011-01-04  44.410000  44.410000  43.200001  35.981472  14066300  AXP   
2      2011-01-05  44.090000  45.160000  43.910000  37.025490  10655300  AXP   
3      2011-01-06  45.380001  45.599998  44.450001  36.770638   6347100  AXP   
4      2011-01-07  44.930000  44.959999  44.000000  36.466484   6387800  AXP   
...           ...        ...        ...        ...        ...       ...  ...   
87575  2022-12-23  86.059998  88.070000  85.769997  87.321243  11171600  DIS   
87576  2022-12-27  87.419998  87.940002  85.959999  85.694084  11561400  DIS   
87577  2022-12-28  86.080002  86.690002  84.070000  83.511292  12399500  DIS   
87578  2022-12-29  85.250000  88.239998  84.970001  86.497742  13045100  DIS   
87579  2022-12-30  85.730003  87.120003  85.230003  86.200081  23231000  DIS   

       d

In [7]:
import pandas as pd
import yfinance as yf

data_df = pd.DataFrame()
num_failures = 0
for tic in DOW_30_TICKER:
      temp_df = yf.download(
          tic, start=TRAIN_START_DATE, end=TEST_END_DATE, proxy=None
      )
      temp_df["tic"] = tic
      print(temp_df)
      if len(temp_df) > 0:
          # data_df = data_df.append(temp_df)
          data_df = pd.concat([data_df, temp_df], axis=0)
      else:
          num_failures += 1
if num_failures == len(DOW_30_TICKER):
    raise ValueError("no data is fetched.")
  # reset the index, we want to use numbers as index instead of dates
data_df = data_df.reset_index()

data_df

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2009-04-01   13.340000   14.640000   13.080000   14.440000   11.463931   
2009-04-02   15.260000   15.300000   14.590000   14.980000   11.892636   
2009-04-03   14.800000   15.330000   14.520000   15.330000   12.170496   
2009-04-06   15.080000   15.270000   14.650000   15.160000   12.035541   
2009-04-07   15.120000   15.380000   14.850000   15.010000   11.916451   
...                ...         ...         ...         ...         ...   
2023-12-22  185.639999  186.369995  184.630005  185.570007  183.374313   
2023-12-26  185.960007  186.179993  185.130005  185.960007  183.759705   
2023-12-27  186.139999  187.130005  185.619995  187.059998  184.846664   
2023-12-28  187.119995  188.070007  187.000000  187.800003  185.577927   
2023-12-29  187.750000  188.300003  186.529999  187.339996  185.123352   

              Volume  tic  
Date     

[*********************100%***********************]  1 of 1 completed


                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2009-04-01    3.717500    3.892857    3.710357    3.881786    3.278072   
2009-04-02    3.933571    4.098214    3.920714    4.025357    3.399315   
2009-04-03    4.078214    4.147500    4.054286    4.142500    3.498239   
2009-04-06    4.105000    4.241071    4.045714    4.230357    3.572432   
2009-04-07    4.161786    4.166786    4.078214    4.107143    3.468380   
...                ...         ...         ...         ...         ...   
2023-12-22  195.179993  195.410004  192.970001  193.600006  192.868149   
2023-12-26  193.610001  193.889999  192.830002  193.050003  192.320221   
2023-12-27  192.490005  193.500000  191.089996  193.149994  192.419830   
2023-12-28  194.139999  194.660004  193.169998  193.580002  192.848206   
2023-12-29  193.899994  194.399994  191.729996  192.529999  191.802170   

               Volume   tic  
Date   

[*********************100%***********************]  1 of 1 completed


...

[*********************100%***********************]  1 of 1 completed


                 Open       High        Low      Close  Adj Close    Volume  \
Date                                                                          
2009-04-01  17.126667  17.683332  17.073334  17.606667  12.571252  67957200   
2009-04-02  17.780001  18.190001  17.733334  17.879999  12.766415  65542800   
2009-04-03  17.900000  18.100000  17.650000  17.933332  12.804498  49566900   
2009-04-06  17.803333  17.916668  17.633333  17.809999  12.716434  46532400   
2009-04-07  17.653334  17.673332  17.333332  17.463333  12.468913  42833100   
...               ...        ...        ...        ...        ...       ...   
2023-12-22  51.816666  52.383331  51.720001  52.216667  51.712521  19405500   
2023-12-26  52.216667  52.330002  52.036667  52.136665  51.633289  11679900   
2023-12-27  52.103333  52.650002  52.053333  52.626667  52.118565  19896000   
2023-12-28  52.590000  52.776669  52.500000  52.523335  52.016228  16776000   
2023-12-29  52.509998  52.693333  52.386665  52.5499

[*********************100%***********************]  1 of 1 completed

                 Open       High        Low      Close  Adj Close    Volume  \
Date                                                                          
2009-04-01  17.920000  18.820000  17.840000  18.790001  16.053432  12933600   
2009-04-02  19.250000  20.500000  19.150000  20.209999  17.266624  25194100   
2009-04-03  19.910000  20.389999  19.780001  20.000000  17.087208  18478000   
2009-04-06  19.780001  20.000000  19.350000  19.620001  16.762556  11280900   
2009-04-07  19.379999  19.379999  19.030001  19.120001  16.335373  11623500   
...               ...        ...        ...        ...        ...       ...   
2023-12-22  91.620003  92.239998  90.750000  91.019997  90.602005   9411500   
2023-12-26  90.839996  91.269997  90.599998  90.949997  90.532326   8086900   
2023-12-27  90.910004  90.940002  90.040001  90.379997  89.964943  11123500   
2023-12-28  90.089996  90.980003  89.970001  90.400002  89.984856   8479600   
2023-12-29  90.120003  90.599998  89.860001  90.2900

,Date,Open,High,Low,Close,Adj Close,Volume,tic
0,2009-04-01,13.340000,14.640000,13.080000,14.440000,11.463931,27701800,AXP
1,2009-04-02,15.260000,15.300000,14.590000,14.980000,11.892636,27296000,AXP
2,2009-04-03,14.800000,15.330000,14.520000,15.330000,12.170496,19195400,AXP
3,2009-04-06,15.080000,15.270000,14.650000,15.160000,12.035541,16065100,AXP
4,2009-04-07,15.120000,15.380000,14.850000,15.010000,11.916451,18546800,AXP
...,...,...,...,...,...,...,...,...
107672,2023-12-22,91.620003,92.239998,90.750000,91.019997,90.602005,9411500,DIS
107673,2023-12-26,90.839996,91.269997,90.599998,90.949997,90.532326,8086900,DIS
107674,2023-12-27,90.910004,90.940002,90.040001,90.379997,89.964943,11123500,DIS
107675,2023-12-28,90.089996,90.980003,89.970001,90.400002,89.984856,8479600,DIS


In [8]:
# Add additional technical indicators
INDICATORS = ['macd',
            'rsi_30',
            'cci_30',
            'dx_30']


FEATURES = ["close", "high", "low", "volume"].extend(INDICATORS)

In [9]:
from finrl.meta.preprocessor.preprocessors import FeatureEngineer
fe = FeatureEngineer(use_technical_indicator=True,
                     tech_indicator_list = INDICATORS,
                     use_turbulence=True,
                     user_defined_feature = False)

processed = fe.preprocess_data(df)
processed = processed.copy()
processed = processed.fillna(0)
processed = processed.replace(np.inf,0)

# here you can see the engineered features are added to each stock day
print(processed)


Successfully added technical indicators
Successfully added turbulence index
             date        open        high         low       close     volume  \
0      2011-01-03   11.630000   11.795000   11.601429    9.939774  445138400   
1      2011-01-03   55.200001   56.279999   55.180000   39.377647    5453300   
2      2011-01-03   43.299999   43.619999   43.110001   35.531181    7633300   
3      2011-01-03   66.150002   66.680000   66.000000   53.034527    8072900   
4      2011-01-03   94.379997   94.809998   94.110001   65.767761    5231500   
...           ...         ...         ...         ...         ...        ...   
87575  2022-12-30  530.000000  530.500000  524.840027  516.237061    1849600   
87576  2022-12-30  206.309998  208.039993  205.699997  204.948471    4159400   
87577  2022-12-30   39.310001   39.689999   39.070000   34.411125   44007200   
87578  2022-12-30   37.240002   37.380001   36.970001   33.196148    5406200   
87579  2022-12-30   47.186668   47.330002   

In [10]:
stock_dimension = len(processed.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 29, State Space: 175


In [11]:

# Split this on the train and trade test data

train = data_split(processed, TRAIN_START_DATE,TRAIN_END_DATE)
test = data_split(processed, TEST_START_DATE,TEST_END_DATE)
print(len(train))
print(len(test))


train.to_csv('train_data.csv')
test.to_csv('test_data.csv')

72993
14587


In [12]:
import pandas as pd
from stable_baselines3.common.logger import configure

from finrl.agents.stablebaselines3.models import DRLAgent
from finrl.config import TRAINED_MODEL_DIR, RESULTS_DIR
from finrl.main import check_and_make_directories
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv

check_and_make_directories([TRAINED_MODEL_DIR])

In [13]:
# Train the data

train = pd.read_csv('train_data.csv')

# If you are not using the data generated from part 1 of this tutorial, make sure 
# it has the columns and index in the form that could be make into the environment. 
# Then you can comment and skip the following two lines.
train = train.set_index(train.columns[0])
train.index.names = ['']

In [14]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 29, State Space: 175


In [15]:
buy_cost_list = sell_cost_list = [COMMISSION_FEE_PERCENT] * stock_dimension
num_stock_shares = [0] * stock_dimension

env_kwargs = {
    "hmax": 100,
    "initial_amount": INITIAL_CASH,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4
}


e_train_gym = StockTradingEnv(df = train, **env_kwargs)

In [16]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


In [18]:
# Train the a2c agent:


agent = DRLAgent(env = env_train)
A2C_model_kwargs = {
                    'n_steps': 5,
                    'ent_coef': 0.005,
                    'learning_rate': 0.0007
                    }

# Lower clip_range makes the stocks flatter, very conservative policy

# TODO try playing around with the number of epochs? n_epochs
# TODO try playing around more with the entropy term, make sure agent does enough exploration during training
# TODO try playing around more with the clip papram here


model_a2c = agent.get_model("a2c",model_kwargs = A2C_model_kwargs)

# set up logger
tmp_path = RESULTS_DIR + '/a2c'
new_logger_a2c = configure(tmp_path, ["stdout", "csv", "tensorboard"])
# Set new logger
model_a2c.set_logger(new_logger_a2c)

{'n_steps': 5, 'ent_coef': 0.005, 'learning_rate': 0.0007}
Using cpu device
Logging to results/a2c


In [24]:
trained_a2c = agent.train_model(model=model_a2c, 
                             tb_log_name='a2c',
                             total_timesteps=5_000)

Original Error: 'obs'
Inner Error: 'reward'
Original Error: 'obs'
Inner Error: 'reward'
Original Error: 'obs'
Inner Error: 'reward'
Original Error: 'obs'
Inner Error: 'reward'
Original Error: 'obs'
Inner Error: 'reward'
Original Error: 'obs'
Inner Error: 'reward'
Original Error: 'obs'
Inner Error: 'reward'
Original Error: 'obs'
Inner Error: 'reward'
Original Error: 'obs'
Inner Error: 'reward'
Original Error: 'obs'
Inner Error: 'reward'
Original Error: 'obs'
Inner Error: 'reward'
Original Error: 'obs'
Inner Error: 'reward'
Original Error: 'obs'
Inner Error: 'reward'
Original Error: 'obs'
Inner Error: 'reward'
Original Error: 'obs'
Inner Error: 'reward'
Original Error: 'obs'
Inner Error: 'reward'
Original Error: 'obs'
Inner Error: 'reward'
Original Error: 'obs'
Inner Error: 'reward'
Original Error: 'obs'
Inner Error: 'reward'
Original Error: 'obs'
Inner Error: 'reward'
Original Error: 'obs'
Inner Error: 'reward'
Original Error: 'obs'
Inner Error: 'reward'
Original Error: 'obs'
Inner Erro

In [25]:
trained_a2c.save(TRAINED_MODEL_DIR + "/agent_a2c")

In [26]:
# Backtest the results


train = pd.read_csv('train_data.csv')
test = pd.read_csv('test_data.csv')

# If you are not using the data generated from part 1 of this tutorial, make sure 
# it has the columns and index in the form that could be make into the environment. 
# Then you can comment and skip the following lines.
train = train.set_index(train.columns[0])
train.index.names = ['']
test = test.set_index(test.columns[0])
test.index.names = ['']

In [27]:
from stable_baselines3 import A2C
from finrl.config import TRAINED_MODEL_DIR

trained_a2c = A2C.load(TRAINED_MODEL_DIR + "/agent_a2c") 

# Here is how to load the model

In [28]:
# Use this to download from collab

# import shutil
# shutil.make_archive('train', 'zip', '../content')